In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import os
from google.cloud import bigquery

In [ ]:
cwd=os.getcwd()
cwd

In [ ]:
os.chdir('..')

In [ ]:
cwd=os.getcwd()
cwd

In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = cwd + '/GCP.json'

In [ ]:
client = bigquery.Client()

In [ ]:
# Set your GCP project details
project_id = "lewagon-statistical-arbitrage"
dataset_id = "FTSE_100_main"  # Replace with your dataset name
table_name = "FTSE100_csv"

# Construct full table path
table_id = f"{project_id}.{dataset_id}.{table_name}"

In [ ]:
query = "SELECT * FROM `lewagon-statistical-arbitrage.FTSE_100_main.FTSE100_csv` ORDER BY Unnamed_0 ASC"
FTSE100 = client.query(query).to_dataframe()

In [ ]:
print(FTSE100)

In [ ]:
daily_weight = pd.read_csv(cwd + "/data/daily_weights.csv")

In [ ]:
from google.cloud import storage
import pandas as pd
import io
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

class Frame:
    def dataset(self):
        print('Fetching and Merging...')
        #os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "lewagon-statistical-arbitrage-ae470f7dcd48.json"
        client = storage.Client()
        bucket = client.get_bucket('stat_arb')
        rename_dict = {
                        'Unnamed: 0': 'date',
                        '1. open': 'open',
                        '2. high': 'high',
                        '3. low': 'low',
                        '4. close': 'close',
                        '5. volume': 'volume'
                    }
        expected_columns = list(rename_dict.values())
        folder_prefix = "FTSE_100/"
        blobs = bucket.list_blobs(prefix=folder_prefix)
        dataframes = []

        for blob in blobs:
            if blob.name.endswith('.csv'):
                content = blob.download_as_string()
                if content.strip():
                    try:
                        df = pd.read_csv(io.StringIO(content.decode('utf-8')), sep=None, engine='python')
                        df = df.rename(columns=rename_dict)
                        df = df.loc[:, df.columns.intersection(expected_columns)]
                        stk=blob.name.split('/')[1]
                        df['source_file']=stk.split('.')[0]
                        dataframes.append(df)

                    except pd.errors.EmptyDataError:
                        print(f"Skipping empty file: {blob.name}")
                else:
                    print(f"Skipping empty file: {blob.name}")

        if dataframes:
            merged_df = pd.concat(dataframes, ignore_index=True)
            print("Merging completed successfully!")
        else:
            print("No valid CSV files found.")
        print('Started prepocessing....')
        merged_df['date'] = pd.to_datetime(merged_df['date'],format='mixed')

        df_modified = merged_df[['date', 'source_file', 'close']].copy()
        df_modified['source_file'] = df_modified['source_file'].str.split('/').str[-1].str.replace('.csv', '')
        df_pivoted = df_modified.pivot_table(index='date', columns='source_file', values='close')
        df_pivoted.reset_index(inplace=True)

        df_latest=df_pivoted[df_pivoted['date']>'2022-01-31']
        #dropping as nan are less than 5 for approx. five stocks and 1 stock is  117 nan
        df_latest.dropna(inplace=True)
        # df_numeric = df_latest.drop(columns=['date'])
        print('Completed prepocessing.')
        return df_latest

data=Frame()
stock_price = data.dataset()

In [ ]:
daily_weight = daily_weight.rename(columns = lambda x : str(x)[:-2])

In [ ]:
daily_weight = daily_weight.rename(columns={'Da': 'date'})

In [ ]:
daily_weight["date"] = pd.to_datetime(daily_weight["date"])

In [ ]:
daily_weight

In [ ]:
daily_weight[['date']]

In [ ]:
stock_aligned = daily_weight[['date']].merge(stock_price, on='date')

In [ ]:
weight_aligned = stock_aligned[["date"]].merge(daily_weight,on="date")

In [ ]:
weight_aligned.set_index("date",inplace=True)

In [ ]:
stock_aligned.set_index("date",inplace=True)

In [ ]:
for name in stock_aligned.columns:
  if name not in weight_aligned.columns:
    stock_aligned.drop(name,axis=1,inplace=True)

In [ ]:
FTSE100.rename(columns={'Unnamed_0': 'Date',"close": "FTSE price"}, inplace=True)

FTSE100.set_index('Date', inplace=True)

FTSE100_close_price = pd.DataFrame(FTSE100["FTSE price"])

FTSE100_close_price

In [ ]:
FTSE100_close_price.index = pd.to_datetime(FTSE100_close_price.index)

In [ ]:
investment_aligned=weight_aligned.join(FTSE100_close_price)

In [ ]:
replication_aligned = weight_aligned.mul(investment_aligned['FTSE price'], axis=0)
replication_aligned.sum(axis=1)

In [ ]:
weight_position = replication_aligned/stock_aligned

In [ ]:
trading_length=90
entry_point=60

In [ ]:
replication=[]
replications_df=pd.DataFrame(columns=range(trading_length), dtype=float)

In [ ]:
for i,r in weight_position.reset_index().iterrows():
  if i < (len(weight_position) - trading_length):
    try_1 = r * stock_aligned.iloc[i:i+trading_length]
    replication_index=pd.DataFrame(try_1.sum(axis=1).reset_index(drop=True))
    replications_df=pd.concat([replications_df,replication_index.T], axis=0)

In [ ]:
replications_df.index=weight_position.index[:-trading_length]

In [ ]:
replications_df.columns=[f'Day {i+1}' for i in range(trading_length)]

In [ ]:
replications_df=replications_df.astype(float)

In [ ]:
FTSE_match=[]
FTSE_match_df=pd.DataFrame(columns=range(trading_length))

In [ ]:
for i,r in weight_position.reset_index().iterrows():
  if i < (len(weight_position) - trading_length):
    FTSE_match=pd.DataFrame(investment_aligned[['FTSE price']].iloc[i:i+trading_length].reset_index(drop=True))
    FTSE_match_df=pd.concat([FTSE_match_df,FTSE_match.T], axis=0)

In [ ]:
FTSE_match_df.index=weight_position.index[:-trading_length]
FTSE_match_df.columns=[f'Day {i+1}' for i in range(trading_length)]

In [ ]:
FTSE_log_returns=np.log(FTSE_match_df/FTSE_match_df.shift(1, axis=1))

In [ ]:
replications_log_returns=np.log(replications_df/replications_df.shift(1, axis=1))

In [ ]:
#assessment after entry_point days

In [ ]:
spread_change=FTSE_log_returns-replications_log_returns

In [ ]:
spread_mean=spread_change.iloc[:,:entry_point-1].mean(axis=1)

In [ ]:
spread_vol=spread_change.iloc[:,:entry_point-1].std(axis=1)

In [ ]:
spread_vol

In [245]:
z_scores_df=pd.DataFrame((spread_change.iloc[:,entry_point]-spread_mean)/spread_vol, columns=['z_score'])

In [246]:
z_scores_df['z_score']>-1

date
2022-07-15     True
2022-07-18     True
2022-07-19     True
2022-07-20     True
2022-07-21    False
              ...  
2024-10-23     True
2024-10-24     True
2024-10-25     True
2024-10-28     True
2024-10-29     True
Name: z_score, Length: 578, dtype: bool

In [247]:
pos_low_threshold=0.5
pos_high_threshold=2
neg_low_threshold=-2
neg_high_threshold=-0.5

In [248]:
for i, r in z_scores_df.iterrows():
    if r['z_score'] > pos_low_threshold:
        if r['z_score'] <pos_high_threshold:
            z_scores_df.loc[i, 'direction']=-1
        else:
            z_scores_df.loc[i,'direction']=0
    elif r['z_score'] > neg_low_threshold:
        if r['z_score']<neg_high_threshold:
            z_scores_df.loc[i, 'direction']=1
        else: z_scores_df.loc[i, 'direction']=0

In [257]:
z_scores_df

,z_score,direction
date,,
2022-07-15,-0.334534,0.0
2022-07-18,0.608562,-1.0
2022-07-19,-0.726178,1.0
2022-07-20,0.708542,-1.0
2022-07-21,-1.073585,1.0
...,...,...
2024-10-23,0.402829,0.0
2024-10-24,-0.208522,0.0
2024-10-25,-0.423520,0.0


In [285]:
z_scores_df['entry']=spread_change.iloc[:,entry_point]

In [286]:
z_scores_df['exit']=spread_change.iloc[:,trading_length-1]

In [288]:
z_scores_df['result']=(-z_scores_df['entry']+z_scores_df['exit'])*z_scores_df['direction']

In [290]:
z_scores_df['result'].sum()

np.float64(1.8357152154801994)